In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch
import os

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
os.getcwd()

'/home/s/dev/robot-whisperer/LLM'

In [28]:
weights_dir = "./weights"
# model_name_or_path = "TheBloke/Mistral-7B-v0.1-AWQ"
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-AWQ"

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True, device=device, cache_dir=weights_dir)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False, device=device, cache_dir=weights_dir)

config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

quant_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

In [26]:
prompt = "Tell me about AI"
prompt_template=f'''{prompt}

'''

print("\n\n*** Generate:")

tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=40,
    max_new_tokens=512
)

print("Output: ", tokenizer.decode(generation_output[0]))




*** Generate:
Output:  <s> Tell me about AI

# AI is a technology that can be used to automate tasks that are repetitive and time-consuming.

AI is a technology that can be used to automate tasks that are repetitive and time-consuming. It can also be used to make decisions based on data, which can help businesses make better decisions.

AI is a technology that can be used to automate tasks that are repetitive and time-consuming. It can also be used to make decisions based on data, which can help businesses make better decisions.

## What is AI?

AI is a technology that can be used to automate tasks that are repetitive and time-consuming. It can also be used to make decisions based on data, which can help businesses make better decisions.

## What are the benefits of AI?

AI is a technology that can be used to automate tasks that are repetitive and time-consuming. It can also be used to make decisions based on data, which can help businesses make better decisions.

## What are the cha

In [24]:
# Inference should be possible with transformers pipeline as well in future
# But currently this is not yet supported by AutoAWQ (correct as of September 25th 2023)
from transformers import pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
    device=device,
)

print(pipe(prompt_template)[0]['generated_text'])

The model 'MistralAWQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'Persimmon

*** Pipeline:
Tell me about AI

I want to be able to create more opportunities for young people.

# 'Michael'

I want to show how alcoholism effects not just the person, but loved ones as well.

# Bethany L

2016-2017: I was nominated for this award because of my work on behalf of homeless people in Derby

## Robyn Wimble

I want to encourage young people to open up about their mental health

# Charlotte Lenman

I want to help young people experiencing low self-esteem.

# Alicia

I want to help others find creative ways to cope with anxiety.

# Connie Chadwick

A campaign to encourage better road safety.

# Jack O

My team and I are encouraging young people to use social media in a positive way.

# Jade C

I helped visually impaired people express how they see the world through photos.

## Megan McNern

I want to challenge the negative uses of anonymous social media sites.

# Kieran S

I restored my local community park.

# Laura F

I'm tackling the misconception that being gay is some